# Week 06

## Some Pandas data exercises

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/io_utils.py
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from data_utils import MinMaxScaler
from io_utils import object_from_json_url

### Dataset Exploration

Let's revisit the house dataset from last homework.

In [ ]:
# Define the location of the json file here
HOUSES_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/LA_housing.json"

houses_info = object_from_json_url(HOUSES_FILE)

#### Some "light" exploration:

Ok. We should now have a list of objects with information about houses in LA.

Let's work with the data to answer the following questions:
- How many houses are in this dataset?
- How many "features" does our dataset have?
- What's the min, max and average price for the houses in the dataset?

In [ ]:
# How many houses are in the dataset?
num_houses = len(houses_info)

# Features:
house_features = houses_info[0].keys()

houses_info[0], num_houses, house_features

In [ ]:
# List of prices

house_prices = []
for h in houses_info:
  house_prices.append(h["value"])

# min price
min_price = min(house_prices)

# max price
max_price = max(house_prices)

# avg price
avg_price = sum(house_prices) // num_houses

min_price, max_price, avg_price

#### More exploring

What if we wanted to get min, max and average values for all of the features?

# 😖

Repeating the code above, can get annoying really quick, but hopefully we can use the `Pandas` library to help.

Once we load our data into a `DataFrame` we can perform many types of calculations.

Here's how we load our data into a `DataFrame`:

In [ ]:
houses_df = pd.DataFrame.from_records(houses_info)

# And to check the first couple of frames
houses_df.head()

Now we have access to each feature by simply indexing the `DataFrame` by the feature's name.

For example, to get a list of all of the prices:

In [ ]:
house_prices = houses_df["value"]

# only print first 5
house_prices.head()

Or a list of all of the ages of the houses:

In [ ]:
house_ages = houses_df["age"]

# only print first 5
house_ages.head()

Or, we can get a sub-section of our original data with only the age and value features:

In [ ]:
house_ages_values = houses_df[["age", "value"]]

# only print first 5
house_ages_values.head()

#### Doing some maths

To get the smallest value of a feature:

In [ ]:
min_price = houses_df["value"].min()
min_price

And the average (or, mean) value for a feature:

In [ ]:
avg_price = houses_df["value"].mean()
avg_price

This gives us a list with the names of the features/columns:

In [ ]:
house_features = list(houses_df.columns)
house_features

It's similar to when we did `houses_info[0].keys()` above.

Either way, we can now iterate over a list of the feature names to calculate min, max and average for each feature:

In [ ]:
for f in house_features:
  print(f)
  print("\tmin:", houses_df[f].min())
  print("\tmax:", houses_df[f].max())
  print("\tavg:", houses_df[f].mean())

We can even ask for the min (or max, or avg) of the whole `DataFrame` and it knows to do it for each column:

In [ ]:
houses_df.min()

#### Average / Mean

The average, or mean, value of a set of numbers is a quantity that represents the center of a collection of numbers. What this means is that we expect about half of the numbers in a collection to be higher than the mean, and the other half to be lower.

The mean of a set of numbers $x_1, x_2, ..., x_n$ is calculated by dividing the sum of the values by the number of values. It's sometimes written like this:

$\displaystyle \mu = \frac{1}{n} \sum_{i=1}^{n} x_i$

which is the same as `sum(X) / len(X)` in Python if `X` is our list of values.

#### Standard Deviation

In addition to the mean, the standard deviation is a measure of the amount of variation in a sequence of numbers.

It's calculated by taking the square root of the average of the squared differences from the mean of the sequence.

In other words, first we calculate the difference between each point and the mean, and square this difference, then sum all of them up, divide by the number of values in the sequence, and finally take the square root:

$\displaystyle \sigma = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(x_{i} - \mu \right)^{2}}$

The standard deviation is a measurement of how close all of the points are to the mean.

<img src="./imgs/std-dev.jpg" width="700px"/>

Unfortunately, there is no short Python code for computing the standard deviation like there is for the average, but the `DataFrame` object has a function for computing it:

In [ ]:
# mean and standard deviation for each feature
for f in houses_df.columns:
  print(f)
  print("\tavg:", houses_df[f].mean())
  print("\tstd:", houses_df[f].std())

We can see that there's greater variability in the number of rooms of a house when compared to the number of bedrooms, but besides that, this isn't very useful yet because we can't really compare the standard deviations from different features that have different units.

We'll see soon how we can use mean and standard deviation to be able to compare, combine, extrapolate values that were measured in different units.

#### Correlation

Let's say we want to figure out if there are any features that correlate strongly with the house prices.

This means figuring out if there are any other features that are a good indication for the value of a house.

We only have a handful of features, so we can always plot them:

In [ ]:
# A lot easier with Pandas DataFrames

for f in house_features:
  if f != "value":
    plt.scatter(houses_df[f], houses_df["value"], alpha=.1)
    plt.xlabel(f)
    plt.ylabel("value")
    plt.show()

#### Covariance Matrix

Instead of looking at the plots there's actually a mathematical way of calculating how much the features of a dataset are "related".

It's called the [covariance](https://en.wikipedia.org/wiki/Covariance), and it measures how much $2$ features change together.

And with our `DataFrame` we can just call:

In [ ]:
houses_df.cov()

This shows us how much each variable is related to every other variable, which will be useful when we start training models and want to reduce the amount of data we need to process, but for now we can just look at the covariances for the `value` feature:

In [ ]:
houses_df.cov()["value"]

If we look at the covariances with the largest magnitudes, we can see that:
- value correlates with the number of rooms: the more rooms, the higher the price of a house.
- value correlates inversely with age: the older the house, the less valuable it is.
- value correlates somewhat with longitude.

#### Standardizing Values

The above calculation gives us some extra information about the data, but there's a problem:

All of the features are in different units. The range of the rooms feature is between $1$ and $17$, while the range of age is $2$ to $55$, and latitude and longitude vary by at most $1$ degree.

Using different units to calculate the covariance can exaggerate how much certain features actually influence each other.

In order to calculate covariance correctly we have to normalize the data in all columns to be "unitless".

One way to do this is to scale each value to be within the range [0, 1] relative to the min and max values of their column.

This is called [MinMax Normalization](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), and we're going to use a library that will do this for us.

In [ ]:
# Scale all the values to be between 0, 1

# This creates a scaler object we can use
min_max_scaler = MinMaxScaler()

# This is how we use the object on our data
houses_df_min_max = min_max_scaler.fit_transform(houses_df)

# Scaled version of DataFrame
houses_df_min_max.head()

In [ ]:
# Now we can finally correctly calculate the covariances
houses_df_min_max.cov()["value"]

We can see that if we scale our features according to the min and max value of each column, longitude is the top feature that correlate with value, followed by number of rooms and then age.

Normalizing is an important step when working with data. This is not only true for calculating covariances, but is even more important when training models with multiple features.

### One more dataset

Let's load...

In [ ]:
# Define the location of the json file here
DIAMONDS_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/diamonds.json"

# Read into Python object
diamonds_info = object_from_json_url(DIAMONDS_FILE)

# Create DataFrame
diamonds_df = pd.DataFrame.from_records(diamonds_info)

diamonds_df.head()

In [ ]:
for f in diamonds_df.columns:
  print(f)
  print("\tmin:", diamonds_df[f].min())
  print("\tmax:", diamonds_df[f].max())
  print("\tavg:", diamonds_df[f].mean())
  print("\tstd:", diamonds_df[f].std())

# 😫

Some of the features aren't numerical, they're words/tags that describe some property of the diamond.

We can still do some analysis. The easiest way is to just drop those columns:

In [ ]:
diamonds_numerical_df = diamonds_df.drop(columns=["cut", "color", "clarity"])
diamonds_numerical_df.head()

#### Now we can get some statistics

In [ ]:
for f in diamonds_numerical_df.columns:
  print(f)
  print("\tmin:", diamonds_numerical_df[f].min())
  print("\tmax:", diamonds_numerical_df[f].max())
  print("\tavg:", diamonds_numerical_df[f].mean())
  print("\tstd:", diamonds_numerical_df[f].std())

# 🤷

Which is fine if all we're doing is taking a look at our data, but once we start training models and doing more detailed analyses we'll want to use all of the data that we have available.

Let's see how to work with non-numerical properties.

#### Encoding

If we look at the [dataset info](https://www.kaggle.com/datasets/shivam2503/diamonds), we'll see that those features have an order:

<img src="./imgs/diamonds.jpg" width="700px"/>

They all have a "worse" and a "best" option.

This means we can assign numbers to them and include them back into our dataset analysis and eventually use them for training models.

In order to do this we'll use an object called [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) from the same library that we used for doing normalization.

First we have to determine all the possible values of each of these features:

In [ ]:
for f in ["cut", "color", "clarity"]:
  print(diamonds_df[f].unique())

Then re-order them from worst to best:

In [ ]:
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

Create the encoder:

In [ ]:
diamond_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

And apply it to our `DataFrame`:

In [ ]:
# Encode the columns
ccc_vals = diamond_encoder.fit_transform(diamonds_df[["cut", "color", "clarity"]].values)

# Put the values back in the original DataFrame
diamonds_df[["cut", "color", "clarity"]] = ccc_vals

Now we can rerun our loop to get statistics:

In [ ]:
for f in diamonds_df.columns:
  print(f)
  print("\tmin:", diamonds_df[f].min())
  print("\tmax:", diamonds_df[f].max())
  print("\tavg:", diamonds_df[f].mean())
  print("\tstd:", diamonds_df[f].std())

Normalize the data, get covariances and plots

In [ ]:
# Scale all the values to be between 0, 1

# This creates a scaler object we can use
min_max_scaler = MinMaxScaler()

# This is how we use the object on our data
diamonds_df_min_max = min_max_scaler.fit_transform(diamonds_df)

# Scaled version of DataFrame
diamonds_df_min_max.head()

In [ ]:
diamonds_df_min_max.cov()["price"]

In [ ]:
# Plot the original data
for f in diamonds_df.columns:
  if f != "price":
    plt.scatter(diamonds_df[f], diamonds_df["price"], alpha=.1)
    plt.xlabel(f)
    plt.ylabel("price")
    plt.show()

### Next time: Regression

In [ ]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

from data_utils import MinMaxScaler
from io_utils import object_from_json_url

# Load Dataset
DIAMONDS_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/diamonds.json"

# Read into DataFrame
diamonds_info = object_from_json_url(DIAMONDS_FILE)
diamonds_df = pd.DataFrame.from_records(diamonds_info)


# Encode
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

diamond_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

ccc_vals = diamond_encoder.fit_transform(diamonds_df[["cut", "color", "clarity"]].values)
diamonds_df[["cut", "color", "clarity"]] = ccc_vals


# Normalize
diamond_scaler = MinMaxScaler()

# This is how we use the object on our data
diamonds_scaled_df = diamond_scaler.fit_transform(diamonds_df)


# Linear Regression
price_model = LinearRegression()

# Drop price and fit the model to predict price
no_price_df = diamonds_scaled_df.drop(columns=["price"])
price_model.fit(no_price_df.values, diamonds_scaled_df["price"].values)

# Run the model on the training data
predict_price = price_model.predict(no_price_df.values)

# Put price predictions back on DataFrame
predicted_scaled_df = no_price_df.assign(price=predict_price)

# Un-normalize the data
predicted_df = diamond_scaler.inverse_transform(predicted_scaled_df)

# Measure error
mean_squared_error(diamonds_df["price"].values, predicted_df["price"].values, squared=False)
